# Livedoorニュースコーパスを用いたニューストピック分類

### Livedoorニュースコーパスのダウンロードと展開

In [1]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2024-07-10 13:37:08--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.1’

ldcc-20140209.tar.g 100%[===================>]   8.44M  15.1MB/s    in 0.6s    

2024-07-10 13:37:09 (15.1 MB/s) - ‘ldcc-20140209.tar.gz.1’ saved [8855190/8855190]



In [2]:
import os
import pandas as pd
import tarfile

# ファイルパスを指定する
tar_file_path = "./content/ldcc-20140209.tar.gz"
extract_folder = "./content/ldcc_data/"

# tar.gzファイルを解凍し、extract_folderに格納する
with tarfile.open(tar_file_path, "r:gz") as tar:
    tar.extractall(path=extract_folder)

### データの読み込み

In [3]:
def read_articles_from_directory(directory_path):
    #ディレクトリーの中から記事のテキストファイルのパスをリストとして読み込む
    files = [f for f in os.listdir(directory_path) if f not in ["LICENSE.txt"]]
    
    articles = []
    for file in files:
        #記事を一つずつ読み込み、url,date,bodyに分け、辞書を作る
        with open(os.path.join(directory_path, file), 
		"r", encoding="utf-8") as f:
            lines = f.readlines()
            articles.append({
                "url": lines[0].strip(),
                "date": lines[1].strip(),
                "body": ''.join(lines[2:]).strip().replace('\n','')
            })
    
    return articles

In [16]:
# 各記事のディレクトリーを取得する
directories = [d for d in os.listdir(extract_folder + "text/") 
	if d not in ["CHANGES.txt", "README.txt"]]

# カテゴリーごとのCSVファイルを作る
csv_file_paths = {}
for directory in directories:
    # Read articles from the category directory
    articles = read_articles_from_directory(extract_folder + "text/" + directory)
    df = pd.DataFrame(articles)

    # Save to CSV
    csv_path = f"./content/csv/{directory}.csv"
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    df.to_csv(csv_path, index=False)
    csv_file_paths[directory] = csv_path

print(csv_file_paths)

{'movie-enter': './content/csv/movie-enter.csv', 'it-life-hack': './content/csv/it-life-hack.csv', 'kaden-channel': './content/csv/kaden-channel.csv', 'topic-news': './content/csv/topic-news.csv', 'livedoor-homme': './content/csv/livedoor-homme.csv', 'peachy': './content/csv/peachy.csv', 'sports-watch': './content/csv/sports-watch.csv', 'dokujo-tsushin': './content/csv/dokujo-tsushin.csv', 'smax': './content/csv/smax.csv'}


In [5]:
categories = [{"label":index,"category":name} for index,name in enumerate(os.listdir("./content/csv/"))]
num_labels = len(categories)
print(categories)



[{'label': 0, 'category': 'topic-news.csv'}, {'label': 1, 'category': 'dokujo-tsushin.csv'}, {'label': 2, 'category': 'livedoor-homme.csv'}, {'label': 3, 'category': 'it-life-hack.csv'}, {'label': 4, 'category': 'peachy.csv'}, {'label': 5, 'category': 'kaden-channel.csv'}, {'label': 6, 'category': 'smax.csv'}, {'label': 7, 'category': 'sports-watch.csv'}, {'label': 8, 'category': 'movie-enter.csv'}]


### モデルの定義

In [6]:
# 必要なモジュールのインポート
!pip install transformers torch 


In [8]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=num_labels)
print(model.classifier)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=9, bias=True)


### データセットの準備

In [23]:
# tokenizerに必要なモジュールのインポート
!pip install fugashi ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 58.7 MB/s eta 0:00:00 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.6/512.6 kB 25.1 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556703 sha256=28af2c2db0c754a8ca50a4917c35426842a6928c86f756ada6091cba648bbf3e
  Stored in directory: /Users/mizuki/Library/Caches/pip/wheels/93/8b/55/dd5978a069678c372520847cf84ba2ec539cb41917c00a2206
Successfully built ipadic


In [24]:
# tokenizerの読み込み
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [9]:
max_length = 512

In [52]:
import random
row_dataset = []
for category in categories:
    print(f"Processing {category['category']}...")
    df = pd.read_csv(f"./content/csv/{category['category']}")
    for i, row in df.iterrows():
        row_dataset.append({
            "text": row["body"],
            "label": category["label"],
            "inputs": tokenizer(row["body"], max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
        })
random.shuffle(row_dataset)

Processing topic-news.csv...
Processing dokujo-tsushin.csv...
Processing livedoor-homme.csv...
Processing it-life-hack.csv...
Processing peachy.csv...
Processing kaden-channel.csv...
Processing smax.csv...
Processing sports-watch.csv...
Processing movie-enter.csv...


In [53]:
# データセットのサマリーを確認
print(f"Number of samples: {len(row_dataset)}")
print(f"Sample 0: {row_dataset[0]}")

Number of samples: 7367
Sample 0: {'text': '嫉妬で彼を束縛していませんか？彼の行動が気になる。携帯が繋がらなければなおのこと。心配が不安になるのは、愛しているがゆえに妄想を働かせるからだ。「彼と連絡がとれないときは、不安で朝まで眠れないんです」という萌さん(27歳)。彼は会社の同期、イベント好きでリーダー的存在だという。「休日になると仲間を集めてバーベキューや飲み会をするんです。カップル以外の男女も参加するのですが、彼はいつも女の子たちにすごく気を使って、必要以上に笑顔を見せたり、髪型とか服装とか私の前でも平気で褒めています。女の子を口説いているわけじゃないとは分かっていても、つい悔しくて眉間に皺が寄ったり不機嫌になったり…。彼からはもう少し大人になれといわれるんですけど」萌さんは彼が好きでたまらないから嫉妬をするのだけど、「実はイベントにも参加する社内の女子社員が彼のことを好きなんです。その子が私のことを『どうしてあんな女が彼女なの』って陰口を叩いているのが耳に入ってしまって」「自分に自信がないから不安でたまらない」と萌さんは嘆く。略奪婚が横行している世の中だ。彼女がいる男に平気で手を出す女性もいる。でも彼女がジタバタしては略奪女の思いのまま。ここは彼のいうように大人になって、余裕の笑みを浮かべ、彼の良きパートナーとして堂々と振る舞ってはいかがだろうか？ 公式の彼女と認められれば社内のこと、そう易々と彼も他の女性の誘いに乗ったりはできないと思うのだが。恋愛をすると彼だけしか見えなくなる。四六時中彼のことを考えて、仕事も趣味も手につかないことは誰もが一度は経験があるだろう。「以前は嫉妬の塊でした」というのはインテリアショップ勤務の礼さん(38歳)。「彼の携帯チェックもしたし、男友達と出かけるときも『誰とどこへ行ったのか』は必ず聞いたし、出張先のホテルに突然押しかけたこともありました」礼さんの異常な嫉妬に彼も辟易していたようだったが、「勤務先の店長が突然病気になり、私が店長代理を任されたんです。今まで彼のことばかり考えていたのに、彼のことを考える暇もなくなりました。仕事をやらざるをえなくなったので」店長が復帰するまでの3か月、礼さんは彼とまともにデートもできなかった。夕食を食べてまた職場に戻ったり、休日も仕事。彼の方が心配を

In [54]:
from torch.utils.data import DataLoader

batch_size = 1
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1

train_dataset_num = int(len(row_dataset) * train_ratio)
valid_dataset_num = int(len(row_dataset) * valid_ratio)
test_dataset_num = len(row_dataset) - train_dataset_num - valid_dataset_num

train_dataset = row_dataset[:train_dataset_num]
valid_dataset = row_dataset[train_dataset_num:train_dataset_num + valid_dataset_num]
test_dataset = row_dataset[train_dataset_num + valid_dataset_num:]

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [62]:
import torch
lr = 1e-5
optimizer_parameters = model.parameters()
optimizer = torch.optim.Adam(optimizer_parameters, lr=lr)

In [73]:
from tqdm import tqdm
model.train()
losses = []
output_labels = []
correct_labels = []
for data in tqdm(train_dataloader):
    optimizer.zero_grad()
    input_ids = data["inputs"]["input_ids"].squeeze(1)
    attention_mask = data["inputs"]["attention_mask"].squeeze(1)
    token_type_ids = data["inputs"]["token_type_ids"].squeeze(1)
    labels = data["label"]

    model_output = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
    logit = model_output.logits
    loss = model_output.loss
    output_label = torch.argmax(logit, dim=1)

    losses.append(loss.item())
    output_labels.append(output_label)
    correct_labels.append(labels)

    loss.backward()
    optimizer.step()

  0%|          | 23/5893 [00:23<1:41:50,  1.04s/it]


KeyboardInterrupt: 

### GPT-2を触ってみる